# Database connectivity with SQLite3

The connect() constructor creates a connection to the SQlite and returns a Connection object.

- We need to create the object of a class called cursor that allows Python code to execute database command in a database session.

- Cursors are created by the connection.cursor() method: they are bound to the connection for the entire lifetime and all the commands are executed in the context of the database session wrapped by the connection.


- fetchall()

Fetch all (remaining) rows of a query result, returning them as a sequence of sequences (e.g. a list of tuples).

- fetchmany(size)

Fetch the next set of rows of a query result, returning a sequence of sequences (e.g. a list of tuples). It will return number of rows that matches to the size
argument

- fetchone()

Fetch the next row of a query result set, returning a single sequence, or None when no more data is available


In [2]:
import sqlite3
conn = sqlite3.connect(r'C:\Users\Rutvik Mehta\Desktop\Python - Programming for everybody\data\emaildb.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Counts')
cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')
fh = open(r'C:\Users\Rutvik Mehta\Desktop\Python - Programming for everybody\data\mbox.txt')
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1].split("@")
    org = email[1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
conn.commit()
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()

iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [3]:
import sqlite3
conn = sqlite3.connect(r'C:\Users\Rutvik Mehta\Desktop\Python - Programming for everybody\data\emaildb.sqlite')
cur = conn.cursor()
cur.execute("CREATE TABLE person (id integer primary key, firstname text, lastname text, dob date)")

In [6]:
from datetime import datetime, date
cur.execute("INSERT INTO person(firstname, lastname, dob) values (?, ?, ?)", ("Joe","Doe","2003-06-25"))
d = date(1723,6,5)
cur.execute("INSERT INTO person(firstname, lastname, dob) values (?, ?, ?)", ("Adam","Smith", d))
conn.commit()

In [9]:
import sqlite3
conn = sqlite3.connect(r'C:\Users\Rutvik Mehta\Desktop\Python - Programming for everybody\data\emaildb.sqlite')
cur = conn.cursor()
cur.execute("CREATE TABLE logs (id integer primary key, message text, timestamp timestamp)")

OperationalError: table logs already exists

In [11]:
cur.execute("INSERT INTO logs(message, timestamp) values (?, ?)", ("message: error",'2012-12-25 23:59:59'))
cur.execute("INSERT INTO logs(message, timestamp) values (?, ?)", ("message: ok", datetime.now()))
conn.commit()

# BLOB

In [1]:
import sqlite3
sqliteConnection = sqlite3.connect('SQLite_Python.db')
cursor = sqliteConnection.cursor()
cursor.execute("DROP TABLE IF EXISTS new_employee")
cursor.execute("""CREATE TABLE new_employee ( id INTEGER PRIMARY KEY,
                       name TEXT NOT NULL, photo BLOB NOT NULL, resume BLOB NOT NULL)""")  
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        blobData = file.read()
    return blobData
def insertBLOB(empId, name, photo, resumeFile):
    try:
          
        print("Connected to SQLite")
        query = """ INSERT INTO new_employee ('id', 'name', 'photo', 'resume') VALUES (?, ?, ?, ?)"""					
        empPhoto = convertToBinaryData(photo)
        resume = convertToBinaryData(resumeFile)
        data_tuple = (empId, name, empPhoto, resume)
        cursor.execute(query, data_tuple)
        sqliteConnection.commit()
        print("Image and file inserted successfully as a BLOB into a table")
        
    except sqlite3.Error as error:
        print("Failed to insert blob data into sqlite table", error)
insertBLOB(1, "Rutvik", r"H:\rutvik_personal\IMAG0544.jpg", r"H:\rutvik_personal\Short Bio.docx")
insertBLOB(2, "Mehta", r"H:\rutvik_personal\GATE.jpg", r"H:\rutvik_personal\rutvik_cv_06_07_2017.pdf")
cursor.close()



Connected to SQLite
Image and file inserted successfully as a BLOB into a table
Connected to SQLite
Image and file inserted successfully as a BLOB into a table


# BLOB_Read

In [2]:
import sqlite3
def writeTofile(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)
    print("Stored blob data into: ", filename, "\n")
def readBlobData(empId):
    try:
        sqliteConnection = sqlite3.connect('SQLite_Python.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")
        sql_fetch_blob_query = """SELECT * from new_employee where id = ?"""
        cursor.execute(sql_fetch_blob_query, (empId,))
        record = cursor.fetchall()
        for row in record:
            print("Id = ", row[0], "Name = ", row[1])
            name  = row[1]
            photo = row[2]
            resumeFile = row[3]
            print("Storing employee image and resume on disk \n")
            photoPath =   name + ".jpg"
            resumePath =   name + "_resume.docx"
            writeTofile(photo, photoPath)
            writeTofile(resumeFile, resumePath)
        cursor.close()
    except sqlite3.Error as error:
        print("Failed to read blob data from sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("sqlite connection is closed")
readBlobData(1)
readBlobData(2)


Connected to SQLite
Id =  1 Name =  Rutvik
Storing employee image and resume on disk 

Stored blob data into:  Rutvik.jpg 

Stored blob data into:  Rutvik_resume.docx 

sqlite connection is closed
Connected to SQLite
Id =  2 Name =  Mehta
Storing employee image and resume on disk 

Stored blob data into:  Mehta.jpg 

Stored blob data into:  Mehta_resume.docx 

sqlite connection is closed
